#README

This is a very long code to run, in the process of making it, we save the code the run it again. But all of the code should be able to run in a single run.

#UNZIP (WE USE GOOGLE COLAB)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/Copy of shopee-product-detection-dataset.zip'

#IMAGE AUGMENTATION

In [ ]:
pip install albumentations

     |████████████████████████████████| 634kB 9.3MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=8c9a0700d88a28c05cb9cf44b1d56a713a2cefa8556a0c4a51e598e1d9d08948
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [ ]:
pip install git+https://github.com/mjkvaak/ImageDataAugmentor

  Cloning https://github.com/mjkvaak/ImageDataAugmentor to /tmp/pip-req-build-268_3jv0
  Running command git clone -q https://github.com/mjkvaak/ImageDataAugmentor /tmp/pip-req-build-268_3jv0
     |████████████████████████████████| 28.2MB 111kB/s 
  Created wheel for ImageDataAugmentor: filename=ImageDataAugmentor-0.0.0-cp36-none-any.whl size=28017 sha256=a5e9a009f96d88a66cd7224a506ab09db353f8dee78f3a9fde696c3e21243bfd
  Stored in directory: /tmp/pip-ephem-wheel-cache-h9011cww/wheels/d9/10/55/6fca35a4072f87d694876d56ece64db3846cf45e1da1c381fe
Successfully built ImageDataAugmentor
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [ ]:
import tensorflow as tf
import keras
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from ImageDataAugmentor.image_data_augmentor import *
import albumentations
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D
from tensorflow.keras.layers import SeparableConv2D,BatchNormalization,MaxPooling2D,Conv2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop

Using TensorFlow backend.


In [ ]:
AUGMENTATIONS = albumentations.Compose([
    #albumentations.Transpose(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.CLAHE(),
    albumentations.Cutout(num_holes=2,max_h_size=70,max_w_size=70,p=0.2),
    albumentations.OneOf([
    albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
    albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=1),
    albumentations.Blur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.Rotate(limit=15),
])

In [ ]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        preprocess_input=None,
        validation_split=0.1)

train_generator = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',
        shuffle=True,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',
        shuffle=False,
        subset='validation')

Found 94869 images belonging to 42 classes.
Found 10523 images belonging to 42 classes.


#DEFINE MODEL 1: INCEPTIONRESNETv2

In [ ]:
base_model = tf.keras.applications.InceptionResNetV2(
    input_shape=None,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling='avg',
    classes=1000,
)

In [ ]:
base_model.trainable=False
model_in = Input(shape=(299,299,3))
x = base_model(model_in,training=False)
x = layers.Dropout(0.1)(x)
model_out = layers.Dense(42,activation='softmax')(x)
model = Model(model_in,model_out)

In [ ]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
dense_13 (Dense)             (None, 42)                64554     
Total params: 54,401,290
Trainable params: 64,554
Non-trainable params: 54,336,736
_________________________________________________________________


#TRAIN MODEL 1: RESNETV2

In [ ]:
#param
LR=0.01
momen=0.95

checkpointer = ModelCheckpoint(filepath='percobaan_kaggle_v3_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
#we dont use LR auto searcher (because the error that we encountered before), so we tune the LR by ourselves

In [ ]:
#log for this epoch is deleted, because we keep train save load our model.
epochs = 10
history = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

In [ ]:
#param
LR=0.001
momen=0.95

checkpointer = ModelCheckpoint(filepath='percobaan_kaggle_v3_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
#we dont use LR auto searcher (because the error that we encountered before), so we tune the LR by ourselves

In [ ]:
#log for this epoch is deleted, because we keep train save load our model.
epochs = 10
history = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

In [ ]:
#param
LR=0.0001
momen=0.95

checkpointer = ModelCheckpoint(filepath='percobaan_kaggle_v3_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])

In [ ]:
#we dont use LR auto searcher (because the error that we encountered before), so we tune the LR by ourselves
epochs = 20
history = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/20
2965/2965 [==============================] - ETA: 0s - loss: 0.9462 - accuracy: 0.7368
Epoch 00001: val_loss improved from inf to 1.05149, saving model to percobaan_kaggle_v3_checkpoint.hdf5
2965/2965 [==============================] - 1727s 582ms/step - loss: 0.9462 - accuracy: 0.7368 - val_loss: 1.0515 - val_accuracy: 0.7172
Epoch 2/20
2965/2965 [==============================] - ETA: 0s - loss: 0.9432 - accuracy: 0.7381
Epoch 00002: val_loss improved from 1.05149 to 1.04810, saving model to percobaan_kaggle_v3_checkpoint.hdf5
2965/2965 [==============================] - 1679s 566ms/step - loss: 0.9432 - accuracy: 0.7381 - val_loss: 1.0481 - val_accuracy: 0.7123
Epoch 3/20
2965/2965 [==============================] - ETA: 0s - loss: 0.9419 - accuracy: 0.7377
Epoch 00003: val_loss improved from 1.04810 to 1.04387, saving model to percobaan_kaggle_v3_checkpoint.hdf5
2965/2965 [==============================] - 1679s 566ms/step - loss: 0.9419 - accuracy: 0.7377 - val_loss: 1.

In [ ]:
#model.save('percobaan_kaggle_v3_sgd_00005_095_40_train.h5')

In [ ]:
#from tensorflow.keras.preprocessing.image import img_to_array
#from tensorflow.keras.models import load_model

#model = load_model('percobaan_kaggle_v3_sgd_00005_095_40_train.h5')

In [ ]:
#param
LR=0.00001
momen=0.95

checkpointer = ModelCheckpoint(filepath='percobaan_kaggle_v3_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])

In [ ]:
epochs = 10
history2 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/10
2965/2965 [==============================] - ETA: 0s - loss: 0.9136 - accuracy: 0.7448
Epoch 00001: val_loss improved from inf to 1.04067, saving model to percobaan_kaggle_v3_checkpoint.hdf5
2965/2965 [==============================] - 1802s 608ms/step - loss: 0.9136 - accuracy: 0.7448 - val_loss: 1.0407 - val_accuracy: 0.7200
Epoch 2/10
2965/2965 [==============================] - ETA: 0s - loss: 0.9121 - accuracy: 0.7459
Epoch 00002: val_loss improved from 1.04067 to 1.02559, saving model to percobaan_kaggle_v3_checkpoint.hdf5
2965/2965 [==============================] - 1846s 622ms/step - loss: 0.9121 - accuracy: 0.7459 - val_loss: 1.0256 - val_accuracy: 0.7244
Epoch 3/10
2965/2965 [==============================] - ETA: 0s - loss: 0.9101 - accuracy: 0.7462
Epoch 00003: val_loss did not improve from 1.02559
2965/2965 [==============================] - 1800s 607ms/step - loss: 0.9101 - accuracy: 0.7462 - val_loss: 1.0419 - val_accuracy: 0.7214
Epoch 4/10
2965/2965 [=======

In [ ]:
model.save('resnetv2_train.h5')

In [ ]:
#starting to fine tuning model

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

model = load_model('resnetv2_train.h5')

In [ ]:
model.trainable = True

In [ ]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
dense_13 (Dense)             (None, 42)                64554     
Total params: 54,401,290
Trainable params: 54,340,746
Non-trainable params: 60,544
_________________________________________________________________


In [ ]:
#param
LR=0.000005
momen=0.95

checkpointer = ModelCheckpoint(filepath='percobaan_resv2_train4_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])

In [ ]:
epochs = 2
history4 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/2
2636/2636 [==============================] - ETA: 0s - loss: 0.9071 - accuracy: 0.7446
Epoch 00001: val_loss improved from inf to 0.90795, saving model to percobaan_resv2_train4_checkpoint.hdf5
2636/2636 [==============================] - 1950s 740ms/step - loss: 0.9071 - accuracy: 0.7446 - val_loss: 0.9079 - val_accuracy: 0.7492
Epoch 2/2
2636/2636 [==============================] - ETA: 0s - loss: 0.7927 - accuracy: 0.7734
Epoch 00002: val_loss improved from 0.90795 to 0.87001, saving model to percobaan_resv2_train4_checkpoint.hdf5
2636/2636 [==============================] - 1938s 735ms/step - loss: 0.7927 - accuracy: 0.7734 - val_loss: 0.8700 - val_accuracy: 0.7628


In [ ]:
model.save('resnetv2_finetuning_train.h5')

In [ ]:
#epochs = 10
#history2 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

In [ ]:
epochs = 8
history4 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/8
2636/2636 [==============================] - ETA: 0s - loss: 0.7199 - accuracy: 0.7921
Epoch 00001: val_loss improved from 0.87001 to 0.84610, saving model to percobaan_resv2_train4_checkpoint.hdf5
2636/2636 [==============================] - 1936s 735ms/step - loss: 0.7199 - accuracy: 0.7921 - val_loss: 0.8461 - val_accuracy: 0.7688
Epoch 2/8
2636/2636 [==============================] - ETA: 0s - loss: 0.6615 - accuracy: 0.8092
Epoch 00002: val_loss improved from 0.84610 to 0.84296, saving model to percobaan_resv2_train4_checkpoint.hdf5
2636/2636 [==============================] - 1935s 734ms/step - loss: 0.6615 - accuracy: 0.8092 - val_loss: 0.8430 - val_accuracy: 0.7744
Epoch 3/8
2636/2636 [==============================] - ETA: 0s - loss: 0.6148 - accuracy: 0.8209
Epoch 00003: val_loss improved from 0.84296 to 0.83357, saving model to percobaan_resv2_train4_checkpoint.hdf5
2636/2636 [==============================] - 1942s 737ms/step - loss: 0.6148 - accuracy: 0.8209 - va

In [ ]:
epochs = 2
history4 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/2
2636/2636 [==============================] - ETA: 0s - loss: 0.2914 - accuracy: 0.9082
Epoch 00001: val_loss improved from inf to 0.87623, saving model to percobaan_resv2_train4_checkpoint.hdf5
2636/2636 [==============================] - 1937s 735ms/step - loss: 0.2914 - accuracy: 0.9082 - val_loss: 0.8762 - val_accuracy: 0.7909
Epoch 2/2
2636/2636 [==============================] - ETA: 0s - loss: 0.2915 - accuracy: 0.9086
Epoch 00002: val_loss did not improve from 0.87623
2636/2636 [==============================] - 1928s 731ms/step - loss: 0.2915 - accuracy: 0.9086 - val_loss: 0.8802 - val_accuracy: 0.7908


In [ ]:
model.save('resnetv2_finetuning2_train.h5')

#DEFINE MODEL 2: NASNETLARGE

In [ ]:
import tensorflow as tf

#checkpointer = ModelCheckpoint(filepath='percobaandensenet.hdf5', verbose=1, save_best_only=True)
#base_model = Xception(weights='imagenet',input_shape=(224,224,3), include_top=False, pooling='avg')
base_model = tf.keras.applications.NASNetLarge(
    input_shape=None,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling='avg',
    classes=1000,
)
#base_model = efn.EfficientNetB7(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')

In [ ]:
base_model.trainable=False
model_in = Input(shape=(331,331,3))
x = base_model(model_in,training=False)
x = layers.Dropout(0.1)(x)
model_out = layers.Dense(42,activation='softmax')(x)
model = Model(model_in,model_out)
model.summary()
#model = Sequential()
#model.add(base_model)
#model.add(layers.Dense(420,activation='relu'))
#model.add(layers.Dropout(0.2))
#model.add(layers.Dense(42,activation='softmax'))"""

In [ ]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        preprocess_input=None,
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(331, 331),
        batch_size=18,
        class_mode='categorical',
        shuffle=True,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(331, 331),
        batch_size=18,
        class_mode='categorical',
        shuffle=False,
        subset='validation')

Found 84331 images belonging to 42 classes.
Found 21061 images belonging to 42 classes.


#TRAIN MODEL 2: NASNETLARGE

In [ ]:
#param
LR=0.01
momen=0.95

checkpointer = ModelCheckpoint(filepath='nasnetlarge_train_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])

In [ ]:
epochs = 5
history = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/5
2636/2636 [==============================] - ETA: 0s - loss: 1.2560 - accuracy: 0.6622
Epoch 00001: val_loss improved from inf to 1.16724, saving model to nasnetlarge_train_checkpoint.hdf5
2636/2636 [==============================] - 1851s 702ms/step - loss: 1.2560 - accuracy: 0.6622 - val_loss: 1.1672 - val_accuracy: 0.6876
Epoch 2/5
2636/2636 [==============================] - ETA: 0s - loss: 1.0797 - accuracy: 0.7046
Epoch 00002: val_loss improved from 1.16724 to 1.13688, saving model to nasnetlarge_train_checkpoint.hdf5
2636/2636 [==============================] - 1834s 696ms/step - loss: 1.0797 - accuracy: 0.7046 - val_loss: 1.1369 - val_accuracy: 0.6944
Epoch 3/5
2636/2636 [==============================] - ETA: 0s - loss: 1.0260 - accuracy: 0.7158
Epoch 00003: val_loss improved from 1.13688 to 1.10202, saving model to nasnetlarge_train_checkpoint.hdf5
2636/2636 [==============================] - 1831s 695ms/step - loss: 1.0260 - accuracy: 0.7158 - val_loss: 1.1020 - va

In [ ]:
#param
LR=0.001
momen=0.95

checkpointer = ModelCheckpoint(filepath='nasnetlarge_train_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])

In [ ]:
epochs = 2
history = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/2
4686/4686 [==============================] - ETA: 0s - loss: 0.8019 - accuracy: 0.7687
Epoch 00001: val_loss improved from inf to 0.99587, saving model to nasnetlarge_train_checkpoint.hdf5
4686/4686 [==============================] - 1910s 407ms/step - loss: 0.8019 - accuracy: 0.7687 - val_loss: 0.9959 - val_accuracy: 0.7308
Epoch 2/2
4686/4686 [==============================] - ETA: 0s - loss: 0.7999 - accuracy: 0.7717
Epoch 00002: val_loss improved from 0.99587 to 0.98411, saving model to nasnetlarge_train_checkpoint.hdf5
4686/4686 [==============================] - 1901s 406ms/step - loss: 0.7999 - accuracy: 0.7717 - val_loss: 0.9841 - val_accuracy: 0.7366


In [ ]:
model.save('nasnetlarge_train3.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

model = load_model('nasnetlarge_train3.h5')

In [ ]:
model.trainable = True

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 331, 331, 3)]     0         
_________________________________________________________________
NASNet (Model)               (None, 4032)              84916818  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4032)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 42)                169386    
Total params: 85,086,204
Trainable params: 84,889,536
Non-trainable params: 196,668
_________________________________________________________________


In [ ]:
#param
LR=0.0001
momen=0.95

checkpointer = ModelCheckpoint(filepath='nasnetlarge_train_checkpoint_finetune.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])

In [ ]:
epochs = 1
history8 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

4686/4686 [==============================] - ETA: 0s - loss: 0.7260 - accuracy: 0.7898
Epoch 00001: val_loss improved from inf to 0.86133, saving model to nasnetlarge_train_checkpoint_finetune.hdf5
4686/4686 [==============================] - 8395s 2s/step - loss: 0.7260 - accuracy: 0.7898 - val_loss: 0.8613 - val_accuracy: 0.7710


In [ ]:
model.save('nasnetlarge_train4_finetuning.h5')

In [ ]:
epochs = 1
history8 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

4686/4686 [==============================] - ETA: 0s - loss: 0.5928 - accuracy: 0.8272
Epoch 00001: val_loss improved from 0.86133 to 0.84497, saving model to nasnetlarge_train_checkpoint_finetune.hdf5
4686/4686 [==============================] - 8394s 2s/step - loss: 0.5928 - accuracy: 0.8272 - val_loss: 0.8450 - val_accuracy: 0.7774


In [ ]:
model.save('nasnetlarge_train_finetuning2.h5')

#DEFINE MODEL 3: EFFICIENTNET B7 

In [ ]:
pip install -U efficientnet

In [ ]:
import efficientnet.tfkeras

base_model = efficientnet.tfkeras.EfficientNetB7(
    input_shape=None,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling='avg',
    classes=1000,
)

258441216/258434480 [==============================] - 3s 0us/step


In [ ]:
base_model.trainable=False
model_in = Input(shape=(299,299,3))
x = base_model(model_in,training=False)
x = layers.Dropout(0.1)(x)
model_out = layers.Dense(42,activation='softmax')(x)
model = Model(model_in,model_out)
model.summary()
#model = Sequential()
#model.add(base_model)
#model.add(layers.Dense(420,activation='relu'))
#model.add(layers.Dropout(0.2))
#model.add(layers.Dense(42,activation='softmax'))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
efficientnet-b7 (Model)      (None, 2560)              64097680  
_________________________________________________________________
dropout (Dropout)            (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 42)                107562    
Total params: 64,205,242
Trainable params: 107,562
Non-trainable params: 64,097,680
_________________________________________________________________


In [ ]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        preprocess_input=None,
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',
        shuffle=True,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',
        shuffle=False,
        subset='validation')

Found 84331 images belonging to 42 classes.
Found 21061 images belonging to 42 classes.


#TRAIN MODEL 3: EFFICIENTNET B7

In [ ]:
#param
LR=0.01
momen=0.95

checkpointer = ModelCheckpoint(filepath='efb7_train_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
    #1

In [ ]:
epochs = 5
history = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/5
2636/2636 [==============================] - ETA: 0s - loss: 1.3249 - accuracy: 0.6447
Epoch 00001: val_loss improved from inf to 1.17280, saving model to efb7_train_checkpoint.hdf5
2636/2636 [==============================] - 1707s 647ms/step - loss: 1.3249 - accuracy: 0.6447 - val_loss: 1.1728 - val_accuracy: 0.6830
Epoch 2/5
2636/2636 [==============================] - ETA: 0s - loss: 1.1353 - accuracy: 0.6906
Epoch 00002: val_loss improved from 1.17280 to 1.13890, saving model to efb7_train_checkpoint.hdf5
2636/2636 [==============================] - 1629s 618ms/step - loss: 1.1353 - accuracy: 0.6906 - val_loss: 1.1389 - val_accuracy: 0.6935
Epoch 3/5
2636/2636 [==============================] - ETA: 0s - loss: 1.0894 - accuracy: 0.7008
Epoch 00003: val_loss improved from 1.13890 to 1.11554, saving model to efb7_train_checkpoint.hdf5
2636/2636 [==============================] - 1577s 598ms/step - loss: 1.0894 - accuracy: 0.7008 - val_loss: 1.1155 - val_accuracy: 0.6972
Ep

In [ ]:
model.save('efb7_train.h5')

In [ ]:
#param
LR=0.0001
momen=0.95

checkpointer = ModelCheckpoint(filepath='efb7_train_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
    #1

In [ ]:
epochs = 2
history = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/2
7028/7028 [==============================] - ETA: 0s - loss: 0.9578 - accuracy: 0.7353
Epoch 00001: val_loss improved from inf to 1.05413, saving model to efb7_train_checkpoint.hdf5
7028/7028 [==============================] - 1725s 245ms/step - loss: 0.9578 - accuracy: 0.7353 - val_loss: 1.0541 - val_accuracy: 0.7159
Epoch 2/2
7028/7028 [==============================] - ETA: 0s - loss: 0.9555 - accuracy: 0.7352
Epoch 00002: val_loss did not improve from 1.05413
7028/7028 [==============================] - 1676s 238ms/step - loss: 0.9555 - accuracy: 0.7352 - val_loss: 1.0563 - val_accuracy: 0.7132


In [ ]:
model.save('efb7_train3.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

model = load_model('efb7_train3.h5')

In [ ]:
model.trainable = True

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
efficientnet-b7 (Model)      (None, 2560)              64097680  
_________________________________________________________________
dropout (Dropout)            (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 42)                107562    
Total params: 64,205,242
Trainable params: 63,894,522
Non-trainable params: 310,720
_________________________________________________________________


In [ ]:
#param
LR=0.00001
momen=0.95

checkpointer = ModelCheckpoint(filepath='efb7_train_finetune_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
    #finetune

In [ ]:
epochs = 1
history2 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

8434/8434 [==============================] - ETA: 0s - loss: 0.8735 - accuracy: 0.7584
Epoch 00001: val_loss improved from inf to 0.95479, saving model to efb7_train_finetune_checkpoint.hdf5
8434/8434 [==============================] - 6791s 805ms/step - loss: 0.8735 - accuracy: 0.7584 - val_loss: 0.9548 - val_accuracy: 0.7456


In [ ]:
model.save('efb7_train_finetune.h5')

In [ ]:
epochs = 1
history2 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

8434/8434 [==============================] - ETA: 0s - loss: 0.8268 - accuracy: 0.7709
Epoch 00001: val_loss improved from 0.95479 to 0.92542, saving model to efb7_train_finetune_checkpoint.hdf5
8434/8434 [==============================] - 6781s 804ms/step - loss: 0.8268 - accuracy: 0.7709 - val_loss: 0.9254 - val_accuracy: 0.7518


In [ ]:
model.save('efb7_train_finetune2.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

model = load_model('efb7_train_finetune2.h5')

In [ ]:
#param
LR=0.000001
momen=0.95

checkpointer = ModelCheckpoint(filepath='efb7_train_finetune_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
    #finetune

In [ ]:
epochs = 1
history2 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

8434/8434 [==============================] - ETA: 0s - loss: 0.8005 - accuracy: 0.7778
Epoch 00001: val_loss improved from inf to 0.92294, saving model to efb7_train_finetune_checkpoint.hdf5
8434/8434 [==============================] - 6771s 803ms/step - loss: 0.8005 - accuracy: 0.7778 - val_loss: 0.9229 - val_accuracy: 0.7524


In [ ]:
model.save('efb7_train_finetune3.h5')

#DEFINE MODEL 4: EFFICIENTNETB6

In [ ]:
pip install -U efficientnet

In [ ]:
import efficientnet.tfkeras

base_model = efficientnet.tfkeras.EfficientNetB6(
    input_shape=None,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling='avg',
    classes=1000,
)

165527552/165527152 [==============================] - 2s 0us/step


In [ ]:
base_model.trainable=False
model_in = Input(shape=(299,299,3))
x = base_model(model_in,training=False)
x = layers.Dropout(0.1)(x)
model_out = layers.Dense(42,activation='softmax')(x)
model = Model(model_in,model_out)
model.summary()
#model = Sequential()
#model.add(base_model)
#model.add(layers.Dense(420,activation='relu'))
#model.add(layers.Dropout(0.2))
#model.add(layers.Dense(42,activation='softmax'))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
efficientnet-b6 (Model)      (None, 2304)              40960136  
_________________________________________________________________
dropout (Dropout)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 42)                96810     
Total params: 41,056,946
Trainable params: 96,810
Non-trainable params: 40,960,136
_________________________________________________________________


In [ ]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        preprocess_input=None,
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',
        shuffle=True,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',
        shuffle=False,
        subset='validation')

Found 17630 images belonging to 9 classes.
Found 4403 images belonging to 9 classes.


#TRAIN MODEL 4: EFFICIENTNET B6

In [ ]:
#param
LR=0.01
momen=0.95

checkpointer = ModelCheckpoint(filepath='efb7_train_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
    #1

In [ ]:
epochs = 5
history = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/5
2636/2636 [==============================] - ETA: 0s - loss: 1.3788 - accuracy: 0.6318
Epoch 00001: val_loss improved from inf to 1.22006, saving model to efb7_train_checkpoint.hdf5
2636/2636 [==============================] - 1485s 563ms/step - loss: 1.3788 - accuracy: 0.6318 - val_loss: 1.2201 - val_accuracy: 0.6724
Epoch 2/5
2636/2636 [==============================] - ETA: 0s - loss: 1.1821 - accuracy: 0.6773
Epoch 00002: val_loss improved from 1.22006 to 1.17889, saving model to efb7_train_checkpoint.hdf5
2636/2636 [==============================] - 1443s 547ms/step - loss: 1.1821 - accuracy: 0.6773 - val_loss: 1.1789 - val_accuracy: 0.6814
Epoch 3/5
2636/2636 [==============================] - ETA: 0s - loss: 1.1321 - accuracy: 0.6871
Epoch 00003: val_loss improved from 1.17889 to 1.15241, saving model to efb7_train_checkpoint.hdf5
2636/2636 [==============================] - 1428s 542ms/step - loss: 1.1321 - accuracy: 0.6871 - val_loss: 1.1524 - val_accuracy: 0.6870
Ep

In [ ]:
#param
LR=0.001
momen=0.95

checkpointer = ModelCheckpoint(filepath='efb6_train_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
    #1

In [ ]:
epochs = 5
history = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/5
2636/2636 [==============================] - ETA: 0s - loss: 1.0351 - accuracy: 0.7148
Epoch 00001: val_loss improved from inf to 1.10633, saving model to efb6_train_checkpoint.hdf5
2636/2636 [==============================] - 1446s 549ms/step - loss: 1.0351 - accuracy: 0.7148 - val_loss: 1.1063 - val_accuracy: 0.7027
Epoch 2/5
2636/2636 [==============================] - ETA: 0s - loss: 1.0271 - accuracy: 0.7188
Epoch 00002: val_loss improved from 1.10633 to 1.10281, saving model to efb6_train_checkpoint.hdf5
2636/2636 [==============================] - 1425s 541ms/step - loss: 1.0271 - accuracy: 0.7188 - val_loss: 1.1028 - val_accuracy: 0.7032
Epoch 3/5
2636/2636 [==============================] - ETA: 0s - loss: 1.0200 - accuracy: 0.7195
Epoch 00003: val_loss improved from 1.10281 to 1.09706, saving model to efb6_train_checkpoint.hdf5
2636/2636 [==============================] - 1427s 541ms/step - loss: 1.0200 - accuracy: 0.7195 - val_loss: 1.0971 - val_accuracy: 0.7054
Ep

In [ ]:
#param
LR=0.0001
momen=0.95

checkpointer = ModelCheckpoint(filepath='efb7_train_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
    #1

In [ ]:
epochs = 5
history = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/5
2636/2636 [==============================] - ETA: 0s - loss: 0.8989 - accuracy: 0.7504
Epoch 00001: val_loss improved from inf to 1.04043, saving model to efb7_train_checkpoint.hdf5
2636/2636 [==============================] - 2297s 871ms/step - loss: 0.8989 - accuracy: 0.7504 - val_loss: 1.0404 - val_accuracy: 0.7199
Epoch 2/5
2636/2636 [==============================] - ETA: 0s - loss: 0.8985 - accuracy: 0.7501
Epoch 00002: val_loss improved from 1.04043 to 1.03745, saving model to efb7_train_checkpoint.hdf5
2636/2636 [==============================] - 2268s 860ms/step - loss: 0.8985 - accuracy: 0.7501 - val_loss: 1.0375 - val_accuracy: 0.7223
Epoch 3/5
2636/2636 [==============================] - ETA: 0s - loss: 0.8991 - accuracy: 0.7493
Epoch 00003: val_loss improved from 1.03745 to 1.03610, saving model to efb7_train_checkpoint.hdf5
2636/2636 [==============================] - 2281s 865ms/step - loss: 0.8991 - accuracy: 0.7493 - val_loss: 1.0361 - val_accuracy: 0.7232
Ep

In [ ]:
#param
LR=0.0001
momen=0.95

checkpointer = ModelCheckpoint(filepath='efb6_train_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
    #1

In [ ]:
epochs = 3
history = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/3
7028/7028 [==============================] - ETA: 0s - loss: 1.0094 - accuracy: 0.7215
Epoch 00001: val_loss improved from inf to 1.10049, saving model to efb6_train_checkpoint.hdf5
7028/7028 [==============================] - 1522s 217ms/step - loss: 1.0094 - accuracy: 0.7215 - val_loss: 1.1005 - val_accuracy: 0.7022
Epoch 2/3
7028/7028 [==============================] - ETA: 0s - loss: 1.0084 - accuracy: 0.7226
Epoch 00002: val_loss did not improve from 1.10049
7028/7028 [==============================] - 1489s 212ms/step - loss: 1.0084 - accuracy: 0.7226 - val_loss: 1.1012 - val_accuracy: 0.7028
Epoch 3/3
7028/7028 [==============================] - ETA: 0s - loss: 1.0081 - accuracy: 0.7233
Epoch 00003: val_loss improved from 1.10049 to 1.09909, saving model to efb6_train_checkpoint.hdf5
7028/7028 [==============================] - 1484s 211ms/step - loss: 1.0081 - accuracy: 0.7233 - val_loss: 1.0991 - val_accuracy: 0.7044


In [ ]:
model.save('efb6_train2.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

model = load_model('efb6_train2.h5')

In [ ]:
model.trainable = True

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
efficientnet-b6 (Model)      (None, 2304)              40960136  
_________________________________________________________________
dropout (Dropout)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 42)                96810     
Total params: 41,056,946
Trainable params: 40,832,514
Non-trainable params: 224,432
_________________________________________________________________


In [ ]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        preprocess_input=None,
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(299, 299),
        batch_size=10,
        class_mode='categorical',
        shuffle=True,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(299, 299),
        batch_size=10,
        class_mode='categorical',
        shuffle=False,
        subset='validation')

Found 84331 images belonging to 42 classes.
Found 21061 images belonging to 42 classes.


In [ ]:
#param
LR=0.00001
momen=0.95

checkpointer = ModelCheckpoint(filepath='efb6_train_finetune_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
    #finetune

In [ ]:
epochs = 1
history2 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

8434/8434 [==============================] - ETA: 0s - loss: 0.7187 - accuracy: 0.8000
Epoch 00001: val_loss improved from inf to 0.89980, saving model to efb6_train_finetune_checkpoint.hdf5
8434/8434 [==============================] - 6932s 822ms/step - loss: 0.7187 - accuracy: 0.8000 - val_loss: 0.8998 - val_accuracy: 0.7611


In [ ]:
model.save('efb6_train_finetune2.h5')

In [ ]:
#param
LR=0.000001
momen=0.95

checkpointer = ModelCheckpoint(filepath='efb6_train_finetune_checkpoint.hdf5', verbose=1, save_best_only=True)
model.compile(
    optimizer=SGD(
    learning_rate=LR,
    momentum=momen,
    name="SGD",
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0,
        reduction="auto",
        name="categorical_crossentropy"),
    metrics=['accuracy'])
    #finetune

In [ ]:
epochs = 1
history2 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

8434/8434 [==============================] - ETA: 0s - loss: 0.6983 - accuracy: 0.8061
Epoch 00001: val_loss improved from inf to 0.90016, saving model to efb6_train_finetune_checkpoint.hdf5
8434/8434 [==============================] - 6910s 819ms/step - loss: 0.6983 - accuracy: 0.8061 - val_loss: 0.9002 - val_accuracy: 0.7579


In [ ]:
epochs = 2
history2 = model.fit(train_generator, epochs=epochs, callbacks =[checkpointer], validation_data=validation_generator)

Epoch 1/2
8434/8434 [==============================] - ETA: 0s - loss: 0.6937 - accuracy: 0.8078
Epoch 00001: val_loss improved from 0.90016 to 0.89328, saving model to efb6_train_finetune_checkpoint.hdf5
8434/8434 [==============================] - 6903s 818ms/step - loss: 0.6937 - accuracy: 0.8078 - val_loss: 0.8933 - val_accuracy: 0.7593
Epoch 2/2
8434/8434 [==============================] - ETA: 0s - loss: 0.6967 - accuracy: 0.8066
Epoch 00002: val_loss did not improve from 0.89328
8434/8434 [==============================] - 6895s 818ms/step - loss: 0.6967 - accuracy: 0.8066 - val_loss: 0.8940 - val_accuracy: 0.7630


In [ ]:
model.save('efb6_train_finetune3.h5')

#ENSEMBLE

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from numpy import dstack

In [ ]:
test_df = pd.read_csv('test.csv')#load path image data test
test_df = test_df.astype('str', copy=False)

In [ ]:
train_df = pd.read_csv('/content/train.csv')#load path image dan kategori data train
train_df = train_df.astype('str', copy=False)

In [ ]:
!pip install deepstack==0.0.9
#installing deepstack ensemble library

In [ ]:
cek = '/content/test/test'#test image data folder path

AUGMENTATIONS = albumentations.Compose([
    #albumentations.Transpose(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.CLAHE(),
    albumentations.Cutout(num_holes=2,max_h_size=70,max_w_size=70,p=0.2),
    albumentations.OneOf([
    albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
    albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=1),
    albumentations.Blur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.Rotate(limit=15),
])

albu_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        preprocess_input=None,
        #validation_split=0.1)
)

test_albu = albu_datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = cek,
    x_col='filename',
    y_col='category',
    #has_ext=False,
    subset='training',
    target_size = (299,299),
    batch_size=32,
    seed=42,
    class_mode = 'categorical',
    shuffle=False,
    validate_filenames = True,
)

Found 12186 validated image filenames belonging to 1 classes.


In [ ]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment = AUGMENTATIONS,
        preprocess_input=None,
        validation_split=0.2)

train_generator_albu = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',
        shuffle=True,
        subset='training')

validation_generator_albu = train_datagen.flow_from_directory(
        '/content/train/train',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',
        shuffle=False,
        subset='validation')

Found 84331 images belonging to 42 classes.
Found 21061 images belonging to 42 classes.


In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import load_model

#load model('model path')
model1 = load_model('resnetv2_finetuning2_train.h5')
model2 = load_model('nasnetlarge_train_finetuning2.h5')
model3 = load_model('efb7_train_finetune3.h5')
model4 = load_model('efb6_train_finetune3.h5')
#model4 = ...
#etc


In [ ]:
from deepstack.base import KerasMember  # For a generic (i.e., Non-Keras Model) check the class `Member`
from deepstack.ensemble import StackEnsemble

In [ ]:
member1 = KerasMember(name='model1',keras_model=model1,train_batches=train_generator_albu,val_batches=validation_generator_albu)
"""member11 = KerasMember(name='model1',keras_model=model1,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member12 = KerasMember(name='model1',keras_model=model1,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member13 = KerasMember(name='model1',keras_model=model1,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member14 = KerasMember(name='model1',keras_model=model1,train_batches=train_generator_albu,val_batches=validation_generator_albu)"""

member2 = KerasMember(name='model2',keras_model=model2,train_batches=train_generator_albu,val_batches=validation_generator_albu)
"""member21 = KerasMember(name='model2',keras_model=model2,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member22 = KerasMember(name='model2',keras_model=model2,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member23 = KerasMember(name='model2',keras_model=model2,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member24 = KerasMember(name='model2',keras_model=model2,train_batches=train_generator_albu,val_batches=validation_generator_albu)"""


member3 = KerasMember(name='model3',keras_model=model3,train_batches=train_generator_albu,val_batches=validation_generator_albu)
"""member31 = KerasMember(name='model3',keras_model=model3,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member32 = KerasMember(name='model3',keras_model=model3,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member33 = KerasMember(name='model3',keras_model=model3,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member34 = KerasMember(name='model3',keras_model=model3,train_batches=train_generator_albu,val_batches=validation_generator_albu)"""

member4 = KerasMember(name='model4',keras_model=model4,train_batches=train_generator_albu,val_batches=validation_generator_albu)
"""member41 = KerasMember(name='model4',keras_model=model4,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member42 = KerasMember(name='model4',keras_model=model4,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member43 = KerasMember(name='model4',keras_model=model4,train_batches=train_generator_albu,val_batches=validation_generator_albu)
member44 = KerasMember(name='model4',keras_model=model4,train_batches=train_generator_albu,val_batches=validation_generator_albu)"""

#member4 = KerasMember(...)

Instructions for updating:
Please use Model.predict, which supports generators.
659/659 [==============================] - 292s 443ms/step


In [ ]:
#Because it was on the edge of the end, we dont use any k fold search to find the best classifier or its param, we just use trial and error method
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from deepstack.ensemble import StackEnsemble
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from deepstack.ensemble import StackEnsemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

stack = StackEnsemble()

"""# 2nd Level Meta-Learner
estimators = [
    #('rf', RandomForestClassifier(verbose=0, n_estimators=200, max_depth=15, n_jobs=20, min_samples_split=30)),
    ('etr', ExtraTreesClassifier(verbose=0, n_estimators=200, max_depth=10, n_jobs=20, min_samples_split=20))
]
# 3rd Level Meta-Learner
clf = StackingClassifier(
    estimators=estimators, final_estimator=make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5))
)
"""

"""stack = StackEnsemble()
stack.model = RandomForestRegressor(verbose=0, n_estimators=200, 
                                  max_depth=15, n_jobs=20, min_samples_split=20)"""


knn = KNeighborsClassifier(n_neighbors=62)
stack = StackEnsemble()
#stack.model = LogisticRegression(random_state=0,n_jobs=4,verbose=1) #this for logistic regressor meta learner
clf = make_pipeline(StandardScaler(),knn) #this for SVC meta-learner
stack.model = clf
#stack.add_members([member1, member2, member3,member4])
stack.add_members([member1, member2, member3, member4])
#stack.add_members([member12, member22, member32,member42])
#stack.add_members([member13, member23, member33,member43])
#stack.add_members([member14, member24, member34,member44])

#stack.add_members([member1, member11, member12,member13,member14, member2, member21, member22, member23, member24, member3, member31, member32, member33, member34, member4, member41, member42, member43, member44])
# pake stack.add_members([member1, member2, member3,member4,member5,...]) sesuai jumlah member dari ensemblenya
stack.fit()
stack.describe(metric = accuracy_score)

Calling predict
model1 - accuracy_score: 0.9856
model2 - accuracy_score: 0.9839
model3 - accuracy_score: 0.9815
model4 - accuracy_score: 0.9827
StackEnsemble accuracy_score: 0.8103


0.8103129006220028

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
from tqdm import tqdm

tta_steps = 10

predictions1 = []

for i in tqdm(range(tta_steps)):
    print(i)
    preds = member1._calculate_predictions(test_image)
    predictions1.append(preds)

prediction1 = np.mean(predictions1, axis=0)

  0%|          | 0/10 [00:00<?, ?it/s]

0
381/381 [==============================] - 99s 261ms/step


 10%|█         | 1/10 [01:40<15:00, 100.05s/it]

1
381/381 [==============================] - 99s 261ms/step


 20%|██        | 2/10 [03:20<13:20, 100.06s/it]

...

 80%|████████  | 8/10 [13:13<03:17, 98.99s/it]

8
381/381 [==============================] - 99s 261ms/step


 90%|█████████ | 9/10 [14:53<01:39, 99.30s/it]

9
381/381 [==============================] - 97s 254ms/step


100%|██████████| 10/10 [16:31<00:00, 99.13s/it]


In [ ]:
from tqdm import tqdm

tta_steps = 10

predictions2 = []

for i in tqdm(range(tta_steps)):
    print(i)
    preds = member2._calculate_predictions(test_image)
    predictions2.append(preds)

prediction2 = np.mean(predictions2, axis=0)

  0%|          | 0/10 [00:00<?, ?it/s]

0
381/381 [==============================] - 178s 467ms/step


 10%|█         | 1/10 [02:58<26:49, 178.87s/it]

1
381/381 [==============================] - 178s 467ms/step


 20%|██        | 2/10 [05:57<23:50, 178.84s/it]

...

 80%|████████  | 8/10 [23:50<05:57, 178.85s/it]

8
381/381 [==============================] - 178s 467ms/step


 90%|█████████ | 9/10 [26:49<02:58, 178.83s/it]

9
381/381 [==============================] - 178s 467ms/step


100%|██████████| 10/10 [29:48<00:00, 178.83s/it]


In [ ]:
from tqdm import tqdm

tta_steps = 10

predictions3 = []

for i in tqdm(range(tta_steps)):
    print(i)
    preds = member3._calculate_predictions(test_image)
    predictions3.append(preds)

prediction3 = np.mean(predictions3, axis=0)

  0%|          | 0/10 [00:00<?, ?it/s]

0
381/381 [==============================] - 150s 394ms/step


 10%|█         | 1/10 [02:30<22:37, 150.85s/it]

1
381/381 [==============================] - 149s 392ms/step


 20%|██        | 2/10 [05:01<20:05, 150.69s/it]

...

 80%|████████  | 8/10 [20:03<05:00, 150.38s/it]

8
381/381 [==============================] - 149s 392ms/step


 90%|█████████ | 9/10 [22:33<02:30, 150.34s/it]

9
381/381 [==============================] - 149s 392ms/step


100%|██████████| 10/10 [25:03<00:00, 150.37s/it]


In [ ]:
from tqdm import tqdm

tta_steps = 10

predictions4 = []

for i in tqdm(range(tta_steps)):
    print(i)
    preds = member4._calculate_predictions(test_image)
    predictions4.append(preds)

prediction4 = np.mean(predictions4, axis=0)

  0%|          | 0/10 [00:00<?, ?it/s]

0
381/381 [==============================] - 149s 392ms/step


 10%|█         | 1/10 [02:30<22:32, 150.26s/it]

1
381/381 [==============================] - 149s 392ms/step


 20%|██        | 2/10 [05:00<20:02, 150.27s/it]

...

 80%|████████  | 8/10 [20:02<05:00, 150.30s/it]

8
381/381 [==============================] - 150s 392ms/step


 90%|█████████ | 9/10 [22:32<02:30, 150.33s/it]

9
381/381 [==============================] - 149s 392ms/step


100%|██████████| 10/10 [25:03<00:00, 150.31s/it]


In [ ]:
prediction_arr = np.hstack((prediction1,prediction2))
prediction_arr = np.hstack((prediction_arr,prediction3))#hstack sesuai jumlah prediction (klo 4 member brarti ada 4 prediction yang harus di hstack)
prediction_arr = np.hstack((prediction_arr,prediction4))

In [ ]:
#predict_test = stack.predict(X=test_image)
#predicting test data
for index, row in test_df.iterrows():
  predict_test_data = stack.predict(X=prediction_arr[index].reshape(1, -1))#stack_predict(stack,cek+'/'+row['filename'])
  row['category'] = predict_test_data[0]#np.argmax(predict_test_data[index])

test_df['category'] = test_df.category.apply(lambda c: str(c).zfill(2))
test_df.to_csv('jawaban.csv', index=False)

In [ ]:
test_df.head()
test_df.to_csv('jawaban.csv', index=False)
#save submission